########## choose stock list (aka market) ##################

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [2]:
## To acquire stock list in NASDAQ ###
ex1 = pd.read_csv('stock_info_us_1.csv')
ex2 = pd.read_csv('stock_info_us_2.csv')
df = pd.DataFrame(ex1['Symbol'].astype('str'))
df = pd.concat([df,pd.DataFrame(ex2['Symbol'].astype('str'))])
df = df.rename(columns={'Symbol' : 'ticker'})
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# df = df.loc[df['ticker'] == 'AMLX']
# df = df.reset_index(drop=True)
df

,ticker
0,AACG
1,AADI
2,AAGR
3,AAGRW
4,AAL
...,...
6755,ZTR
6756,ZTS
6757,ZUO
6758,ZVIA


### Acquire informations from yfinance function >> .info

In [3]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [4]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare',
    'profitMargins',
    'trailingPE'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


#### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_us.csv',index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5ED?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5ED&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EE&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EF&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ACP%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&co

############################################################################

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_us.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Industrials' 'Technology' 'Consumer Cyclical' 'Healthcare' 'Utilities'
 'Consumer Defensive' 'Communication Services' 'Financial Services'
 'Energy' 'Basic Materials' 'Real Estate']
industry ['Building Products & Equipment' 'Consumer Electronics' 'Travel Services'
 'Biotechnology' 'Software - Infrastructure'
 'Semiconductor Equipment & Materials' 'Business Equipment & Supplies'
 'Software - Application' 'Semiconductors' 'Medical Care Facilities'
 'Electrical Equipment & Parts' 'Utilities - Regulated Electric'
 'Education & Training Services' 'Conglomerates'
 'Medical Instruments & Supplies' 'Drug Manufacturers—Specialty & Generic'
 'Electronic Components' 'Computer Hardware' 'Software—Application'
 'Software—Infrastructure' 'Entertainment' 'Drug Manufacturers—General'
 'Capital Markets' 'Furnishings, Fixtures & Appliances' 'Internet Retail'
 'Food Distribution' 'Internet Content & Information' 'Oil & Gas E&P'
 'Trucking' 'Home Improvement Retail' 'Specialty Retail' 'Thermal Co

In [6]:

### Next I perform calculations for MF_ROC and MF_EY ###
# represents_earning = 'Operating Income'
represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,AAON,2024-08-20,Building Products & Equipment,Industrials,7.403666e+09,0.035,0.15605,40.775784,1.010716e+09,1.413190e+08,2.431950e+08,2.426760e+08,2024-06-30,0.279728,0.032848
1,AAPL,2024-08-20,Consumer Electronics,Technology,3.473966e+12,4.065,0.26441,34.382040,3.316120e+11,1.316240e+11,1.216250e+11,1.205940e+11,2024-06-30,0.608161,0.035010
2,ABNB,2024-08-20,Travel Services,Consumer Cyclical,6.636993e+10,17.790,0.46111,16.209524,2.632000e+10,1.582800e+10,1.971000e+09,1.598000e+09,2024-06-30,0.187857,0.029697
3,ACAD,2024-08-20,Biotechnology,Healthcare,2.170921e+09,3.020,0.03432,82.947370,9.140990e+08,3.378510e+08,2.281500e+07,2.281500e+07,2024-06-30,0.039592,0.010509
4,ACIW,2024-08-20,Software - Infrastructure,Technology,5.901930e+09,1.500,0.12011,28.147057,3.364672e+09,1.044121e+09,3.108230e+08,2.975010e+08,2024-06-30,0.133944,0.052665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882,ZBH,2024-08-20,Medical Devices,Healthcare,2.838513e+10,2.063,0.13251,23.334028,2.152000e+10,3.608400e+09,1.224200e+09,1.507600e+09,2024-06-30,0.068347,0.043128
1883,ZIP,2024-08-20,Staffing & Employment Services,Industrials,9.384733e+08,5.268,0.05622,32.607143,6.652730e+08,7.577100e+07,7.541800e+07,5.447000e+07,2024-06-30,0.127935,0.080362
1884,ZTO,2024-08-20,Integrated Freight & Logistics,Industrials,1.316705e+10,24.325,0.21588,14.166666,8.987536e+10,2.326055e+10,1.087243e+10,1.032418e+10,2024-03-31,0.163213,0.825731
1885,ZTS,2024-08-20,Drug Manufacturers—Specialty & Generic,Healthcare,8.877285e+10,3.474,0.26293,36.243614,1.416500e+10,1.797000e+09,3.129000e+09,3.214000e+09,2024-06-30,0.252992,0.035247


In [7]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [8]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [9]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
298,JFIN,2024-08-20,Internet Content & Information,Communication Services,769481792.0,2.678,0.22182,1.801187,5.160597e+09,2.004158e+09,1.299660e+09,1.299660e+09,2024-03-31,0.411749,1.689007,1319.0,1359.0,2678.0


In [10]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,JFIN,2024-08-20,Internet Content & Information,Communication Services,7.694818e+08,2.678,0.22182,1.801187,5.160597e+09,2.004158e+09,1.299660e+09,1.299660e+09,2024-03-31,0.411749,1.689007,1319.0,1359.0,2678.0
1,RMNI,2024-08-20,Software—Application,Technology,1.087560e+08,1.480,0.03801,9.722221,3.673690e+08,3.032360e+08,3.000900e+07,3.953700e+07,2024-06-30,0.467918,0.275930,1331.0,1328.0,2659.0
2,GDEN,2024-08-20,Resorts & Casinos,Consumer Cyclical,1.271011e+09,3.130,0.33072,3.431131,1.181221e+09,1.693220e+08,4.187260e+08,6.473400e+07,2024-06-30,0.413802,0.329443,1321.0,1333.0,2654.0
3,PROC,2024-08-20,Drug Manufacturers—Specialty & Generic,Healthcare,4.715110e+08,0.174,0.12606,3.882353,4.985540e+08,2.029300e+08,1.289630e+08,3.238400e+07,2023-09-30,0.436240,0.273510,1326.0,1327.0,2653.0
4,NVO,2024-08-20,Biotechnology,Healthcare,5.973099e+11,14.055,0.34844,45.538720,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.642715,0.195269,1349.0,1296.0,2645.0
5,KARO,2024-08-20,Software—Application,Technology,4.412004e+08,32.242,0.18699,24.965517,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.329119,2.618490,1274.0,1360.0,2634.0
6,REVG,2024-08-20,Farm & Heavy Construction Machinery,Industrials,1.754983e+09,0.736,0.09413,6.978824,1.330400e+09,5.185000e+08,3.462000e+08,1.043000e+08,2024-04-30,0.426407,0.197267,1323.0,1301.0,2624.0
7,BWMX,2024-08-20,Specialty Retail,Consumer Cyclical,5.423300e+09,11.364,0.08863,8.197675,1.101829e+10,3.860136e+09,2.397458e+09,2.377988e+09,2024-06-30,0.334927,0.442066,1280.0,1343.0,2623.0
8,SIGA,2024-08-20,Drug Manufacturers—Specialty & Generic,Healthcare,6.397935e+08,1.499,0.48551,8.923077,1.926586e+08,2.137805e+07,1.027231e+08,1.027231e+08,2024-06-30,0.599736,0.160557,1347.0,1269.0,2616.0
9,OMAB,2024-08-20,Airports & Air Services,Industrials,3.485830e+10,4.169,0.33793,11.801075,2.590749e+10,4.752860e+09,7.834555e+09,7.527290e+09,2024-06-30,0.370347,0.224754,1297.0,1310.0,2607.0


In [11]:
df_sorted.loc[df_sorted['ticker'] == 'ASO']

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
108,ASO,2024-08-20,Specialty Retail,Consumer Cyclical,5.326394e+09,5.243,0.08171,8.5,4.676713e+09,879858000.0,709207000.0,677855000.0,2024-01-31,0.186788,0.13315,1043.0,1226.0,2269.0


In [12]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,TLK,2024-08-20,Telecom Services,Communication Services,6.999801e+13,283.892,0.15515,12.793334,2.859920e+14,8.317300e+13,4.434200e+13,4.457200e+13,2024-06-30,0.218628,0.633475,1134.0,1351.0,2485.0
1,TSM,2024-08-20,Semiconductors,Technology,3.557613e+12,79.010,0.37854,31.108156,5.787891e+12,1.026180e+12,1.082195e+12,1.023834e+12,2024-06-30,0.227270,0.304191,1146.0,1331.0,2477.0
2,DDS,2024-08-20,Department Stores,Consumer Cyclical,5.144224e+09,66.074,0.09782,8.613523,3.937156e+09,1.130290e+09,9.238860e+08,8.957790e+08,2024-04-30,0.329152,0.179597,1275.0,1288.0,2563.0
3,ASR,2024-08-20,Airports & Air Services,Industrials,8.970210e+10,49.990,0.42797,14.772067,7.299319e+10,4.898742e+09,1.674405e+10,1.580910e+10,2024-03-31,0.245894,0.186663,1186.0,1291.0,2477.0
4,PDD,2024-08-20,Internet Retail,Consumer Cyclical,5.999161e+11,43.947,0.26927,19.830710,3.778261e+11,1.513980e+11,9.543905e+10,7.774340e+10,2024-03-31,0.421498,0.159087,1322.0,1267.0,2589.0
5,KARO,2024-08-20,Software—Application,Technology,4.412004e+08,32.242,0.18699,24.965517,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.329119,2.618490,1274.0,1360.0,2634.0
6,IDCC,2024-08-20,Software—Application,Technology,3.122041e+09,30.257,0.37991,13.543631,1.636295e+09,6.993470e+08,3.677850e+08,3.166220e+08,2024-06-30,0.392535,0.117803,1308.0,1183.0,2491.0
7,AMR,2024-08-20,Coking Coal,Basic Materials,2.775953e+09,25.826,0.13524,6.998533,2.492952e+09,3.086540e+08,5.282050e+08,5.430350e+08,2024-06-30,0.241819,0.190279,1181.0,1294.0,2475.0
8,PAC,2024-08-20,Airports & Air Services,Industrials,1.151602e+11,24.907,0.37654,17.117468,7.331772e+10,1.631331e+10,1.559022e+10,1.458257e+10,2024-06-30,0.273491,0.135378,1222.0,1228.0,2450.0
9,JBL,2024-08-20,Electronic Components,Technology,1.296782e+10,21.887,0.04625,9.760218,1.744900e+10,1.142300e+10,1.990000e+09,1.520000e+09,2024-05-31,0.330236,0.153457,1276.0,1257.0,2533.0
